Telco Customer Churn Prediction Project

## 1. Veri Yükleme ve Ön İnceleme (EDA)

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

df = pd.read_csv("../data/data.csv")

## Veri Ön İşleme (Preprocessing)  

In [32]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
#df.head()


In [33]:
df['TotalCharges'] = df['TotalCharges'].fillna(0)
df.isnull().sum()
print(df['TotalCharges'].dtype)


float64


In [34]:
df['Churn'].value_counts(normalize=True)


Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

Elimizde Soyle Bir Tablo Var:  
-%73 Mutlu Müşteri (No): Kalmışlar.  
-%26 Küskün Müşteri (Yes): Gitmişler.  
-Bizim burada yapmamız gereken şeylerden en önemlisi modelimizi eğitirken doğruluk oranımızın /73 üstü olması gerekiyor.  
->Çünkü biz hiçbir şey yapmasak model  Yes / No dese %73 tutturacak zaten.  

In [35]:
df.drop('customerID', axis=1,inplace=True)

In [36]:
df.groupby('gender')['Churn'].value_counts(normalize=True)

gender  Churn
Female  No       0.730791
        Yes      0.269209
Male    No       0.738397
        Yes      0.261603
Name: proportion, dtype: float64

Özellikleri seçerken cinsiyet faktörü etkiler mi etkilemez mi diye düşünücek olursak bir üst  tabloya bakarak karar verebilirsiniz.

In [37]:
df.drop('gender',axis=1,inplace=True)

In [38]:
df.groupby('Churn')[['tenure','MonthlyCharges']].mean(numeric_only=True)


,tenure,MonthlyCharges
Churn,,
No,37.569965,61.265124
Yes,17.979133,74.441332


Gidenlerin faturası, kalanlara göre %20 daha pahalı.

In [39]:
df.groupby('Churn')['Contract'].value_counts(normalize=True)

Churn  Contract      
No     Month-to-month    0.429068
       Two year          0.318322
       One year          0.252609
Yes    Month-to-month    0.885500
       One year          0.088818
       Two year          0.025682
Name: proportion, dtype: float64

Gidenlerin (Yes) %88'i "Month-to-month" (Taahhütsüz) sözleşme kullanıyormuş.  
Gidenlerin sadece %2'si "Two year" (2 Yıllık) sözleşmeliymiş.  

->Modelimizi geliştirme aşamasına geçmeden önce şu analizi iyi yapmış olmamız gerekiyor..  
1.Yeni Gelmiş Olması  
2.Faturaları yüksek gelmiş  
3.Taahhütsüz olması  
Bizim şirketimizden müşterinin gitmesine neden olan sebeblerdir.  


In [40]:
new_df = df.replace({"Yes":1,"No":0})
df.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Bilgisayarımız Yes ve no dan anlamadığı için tüm 'Yes'=1 yaptık 'No'=0 yaptık..


In [41]:
new_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SeniorCitizen     7043 non-null   int64  
 1   Partner           7043 non-null   object 
 2   Dependents        7043 non-null   object 
 3   tenure            7043 non-null   int64  
 4   PhoneService      7043 non-null   object 
 5   MultipleLines     7043 non-null   object 
 6   InternetService   7043 non-null   object 
 7   OnlineSecurity    7043 non-null   object 
 8   OnlineBackup      7043 non-null   object 
 9   DeviceProtection  7043 non-null   object 
 10  TechSupport       7043 non-null   object 
 11  StreamingTV       7043 non-null   object 
 12  StreamingMovies   7043 non-null   object 
 13  Contract          7043 non-null   str    
 14  PaperlessBilling  7043 non-null   object 
 15  PaymentMethod     7043 non-null   str    
 16  MonthlyCharges    7043 non-null   float64
 17  TotalC

In [42]:
new_df = pd.get_dummies (new_df, drop_first=True)  #One-Hot Encoding

drop_first=True gereksiz olan sütünü atıcaktır 3 sütün yerine  2 sutunla idare edicektir.  


In [43]:
new_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   int64  
 1   tenure                                 7043 non-null   int64  
 2   MonthlyCharges                         7043 non-null   float64
 3   TotalCharges                           7043 non-null   float64
 4   Partner_1                              7043 non-null   bool   
 5   Dependents_1                           7043 non-null   bool   
 6   PhoneService_1                         7043 non-null   bool   
 7   MultipleLines_1                        7043 non-null   bool   
 8   MultipleLines_No phone service         7043 non-null   bool   
 9   InternetService_DSL                    7043 non-null   bool   
 10  InternetService_Fiber optic            7043 non-null   bool   
 11  OnlineSecurity_

In [44]:
y = new_df['Churn_1']
X = new_df.drop('Churn_1',axis=1)

In [45]:
from sklearn.model_selection import train_test_split
X_train , X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


Not: Veriyi önce ham haliyle (Scaling olmadan) modele sokup, başarısızlığı göreceğiz. Ardından Standartlaştırma (Scaling) yapıp farkı inceleyeceğiz.

## 1. Model:  LogisticRegression

In [46]:
from sklearn.linear_model import LogisticRegression
log_R = LogisticRegression(max_iter=5000,class_weight='balanced') #Modeli cezalandırdık class_weight ile bunu daha sonradan ekledim
log_R.fit(X_train,y_train)
y_pred = log_R.predict(X_test)

In [47]:
from sklearn.metrics import accuracy_score
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix (y_test,y_pred)
print(cm)



0.7501774308019872
[[747 289]
 [ 63 310]]


**Doğru Bildiklerimiz (Yeşil Alan):**
    * **747 Kişi (True Negative):** Model "Bu müşteri sadık, gitmez" dedi ve gerçekten de gitmediler.  
    * **310 Kişi (True Positive):** Model "Bu müşteri gidici, dikkat!" dedi ve gerçekten de bu kişiler churn (terk) etti. **(Başarımız burada saklı!)**  

2.  **Yanıldıklarımız (Kırmızı Alan):**
    * **289 Kişi (False Positive - Yanlış Alarm):** Model bu kişilere "Gidecek" dedi ama gitmediler.  
        * *İş Yorumu:* Bu kişilere boşuna indirim teklif edebiliriz, şirket biraz maliyet kaybeder ama müşteri kaybetmez.  
    * **63 Kişi (False Negative - Kaçan Balık):** Model bu kişilere "Gitmez, rahat ol" dedi ama müşteriler sessizce gitti.  
        * *İş Yorumu:* En tehlikeli hatamız bu. Ama sayıca az (63 kişi), bu da modelin "Gidenleri Yakalama (Recall)" konusunda gayet iyi olduğunu gösterir.  
**SONUÇ:** Toplam 373 giden müşterinin (310 + 63), **310 tanesini (%83)** başarıyla tespit ettik! 🚀  

İlk görevimiz özelliklerin değerlerini 0-1 arasında sıkıştırmak.    
Nedenini sorucak olursanız modelimiz yanılgıya düşüyor.  

In [48]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


->Senior gibi mühendislik takasını (trade-off) yaptık başarımız azaldı ama elde gözle görülür bir verimiz var.  

## 2. Model: StandardScaler Haliyle LogisticRegression

In [49]:
log_R = LogisticRegression(max_iter=3000,class_weight='balanced')
log_R.fit(X_train,y_train)
y_pred = log_R.predict(X_test)
accuracy_1 = accuracy_score(y_test,y_pred)
cm_1 = confusion_matrix(y_test,y_pred)
print(accuracy_1)
print(cm_1)


0.7480482611781405
[[746 290]
 [ 65 308]]


->Fazladan  +85 Kişiyi Gördü ve Yakaladi.  
->Kaçanlar Azaldı Eskiden 151  idi şimdi 65.  
->a12'da biraz sorun oluştu model o kadar dedektif oldu ki gitmeyene de gidicek gözüyle baktı.  

Şimdi burda duralım bizim baseline'dan ne farkımız var!    
->Burda önemli olan düşünce baseline tahmin yaparken %50 %50 tahmin yapar ve böyle olursa da şirketten kaça müşteri tahmini = 0 olur.  
-->Bu durumda bizim hiçbir işimize yaramaz.  
--->Bizim modelimiz 308 tane kaçan veya kaçıcak olan müşteri yakaladı başarı oranımız %74.8.  
---->Şirket çalışanı olarak düşünücek olursak 308 kişiye %10-%20 arasında bir indirim uygulayarak şirketimize müşteriyi tekrar kazandırabiliriz.  
Tabloyu incelemek isteyenler aşağıdaki tabloya iyice bakabilir.

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.92      0.72      0.81      1036
        True       0.52      0.83      0.63       373

    accuracy                           0.75      1409
   macro avg       0.72      0.77      0.72      1409
weighted avg       0.81      0.75      0.76      1409



"recall" -> True değerine bakalım 100 müşteriden 83 ünü doğru tahmin ettik.    
Baseline modelimiz bizim 83 olarak bulduğumuz değeri  0 olarak tahmin yapar.  

Bakalım sözleşmesini iptal edip gidenler neden gitti?  

In [51]:
log_R.coef_

array([[ 0.05749907, -1.25252374, -0.85436894,  0.61394192,  0.01805553,
        -0.06853581, -0.00343904,  0.16676418,  0.00343904, -0.29025887,
         0.39658621, -0.13246729, -0.14344377, -0.01948748, -0.14344377,
         0.03454164, -0.14344377, -0.09287449, -0.14344377,  0.22013688,
        -0.14344377,  0.27389903, -0.14344377, -0.28193482, -0.64889622,
         0.15372694, -0.02692245,  0.16106539, -0.01546857]])

En yüksek 'POZİTİF' özellik ->Sözleşmesine son vericek sebebdir.
En düşük  'NEGATİF' özellik->Bizim şirketimizle devam etmek isteyeceklerin sebebidir.

In [52]:
pd.Series(data=log_R.coef_[0],index=X.columns).sort_values()

tenure                                  -1.252524
MonthlyCharges                          -0.854369
Contract_Two year                       -0.648896
InternetService_DSL                     -0.290259
Contract_One year                       -0.281935
StreamingTV_No internet service         -0.143444
OnlineBackup_No internet service        -0.143444
OnlineSecurity_No internet service      -0.143444
DeviceProtection_No internet service    -0.143444
StreamingMovies_No internet service     -0.143444
TechSupport_No internet service         -0.143444
OnlineSecurity_1                        -0.132467
TechSupport_1                           -0.092874
Dependents_1                            -0.068536
PaymentMethod_Credit card (automatic)   -0.026922
OnlineBackup_1                          -0.019487
PaymentMethod_Mailed check              -0.015469
PhoneService_1                          -0.003439
MultipleLines_No phone service           0.003439
Partner_1                                0.018056


-->'TotalCharges' özelliğimiz arttıkça müşterinin  sözleşmesini iptal ettiğini görebiliyoruz.  
-->'Tenure' ise şirkete en sadık olan insanlardır.


Aşağıdak kod grubumuza bakıcak olursak RassalAğaçlar Sınıflandırması kullanılmıştır.    
Başarı oranımızı kıyaslicak olursanız LogReg Sınıflandırmasına göre daha başarısız olduğunu görücek olursunuz.  


In [53]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced')
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
accuracy_2 = accuracy_score(y_test,y_pred)
cm_2 = confusion_matrix(y_test,y_pred)
print(accuracy_2)
print(cm_2)

0.7920511000709723
[[945  91]
 [202 171]]


## 3. Model: Kısıtlandırılmış RandomForestClassifier

In [54]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced')
rfc.fit(X_train,y_train)
y_olasiliklar = rfc.predict_proba(X_test)
yeni_tahmin = (y_olasiliklar[:, 1] > 0.30)
accuracy_2 = accuracy_score(y_test,yeni_tahmin)
cm_2 = confusion_matrix(y_test,yeni_tahmin)
print(accuracy_2)
print(cm_2)

0.7700496806245565
[[814 222]
 [102 271]]


!!815 sadık müşteri tahmin etti.  
->2.satır 1.sütüna bakıcak olursanız yaklaşık 100 kişiyi şirketimize geri kazandırabiliriz.   
->1.satır 2.sütüna bakıcak olursanız da    
!!Yaklaşık 130 tane masum müşteriye de "Sen gideceksin galiba" dedi.  
Şirket için için sorun değil %10 luk indirim tanımladık .Her ihtimalle kısıtlanmamış RandomForest'dan daha iyidir.    

## 4. Model: KNN

In [55]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test,y_pred)
print(accuracy)
print(cm)

0.7643718949609652
[[885 151]
 [181 192]]


**Şu ana kadar hangisi daha iyiydi?**
KNN algoritması çalışmış ama sonuçlar Random Forest'ın gerisinde kaldı.    
Bu durumda halen en iyi sınıflandırma çeşidimiz -->>LogisticRegression  
Gidenleri yakalama konusunda (Recall) Logistic Regression liderliğini koruyor.  


## 5. Model: Gradient Boosting (XGBoost)